In [1]:
import tensorflow as tf
import pandas as pd
import random
import numpy as np
from matplotlib.pyplot import *
from tensorflow.python.tools import freeze_graph

In [2]:
import sklearn.preprocessing

In [3]:
from PIL import Image

In [4]:
def getNextBatch(sizeOfBatch =10,maxNumberOfCircles=3):
    imArray = []
    label_oneHot=np.zeros((sizeOfBatch,maxNumberOfCircles))
    for i in range(sizeOfBatch):
        numberOfCircles= random.randint(1,maxNumberOfCircles)
        imageNumber = random.randint(1,9999)
        im = Image.open(str(numberOfCircles)+"Circle/"+str(imageNumber)+".png")
        imNp = np.array(im.getdata())
        #imNp = imNp.astype(float)
        imArray = np.append(imArray,imNp)
        label_oneHot[i,numberOfCircles-1] = 1
        #imshow(imNp.resize(50,50))

    imArray = imArray/127.5 -1
    imArray.resize(sizeOfBatch,50,50)
    #imshow((imArray[1]+1)*127.5)
    #print(label_oneHot[1])
    imArray.resize(sizeOfBatch,50,50,1)
    return imArray,label_oneHot

In [5]:
def CNNmodel(image,maxNumberOfCircles=3):
    
    with tf.variable_scope("model", reuse=tf.AUTO_REUSE):
        
        hidden1 = tf.layers.conv2d(image, 64, 2)
        hidden1 = tf.nn.leaky_relu(hidden1)
        
        hidden2 = tf.layers.conv2d(hidden1,32,3)
        hidden2 = tf.nn.tanh(hidden2)
        
        hidden2 = tf.contrib.layers.flatten(hidden2) 
        
        hidden3 = tf.layers.dense(inputs=hidden2, units=512)
        hidden3 = tf.nn.tanh(hidden3)
        
        hidden4 = tf.layers.dense(hidden3, maxNumberOfCircles)
        hidden4 = tf.nn.sigmoid(hidden4)
        
        return hidden4
        

In [6]:
img_ph = tf.placeholder(tf.float32,shape=[None,50,50,1])
label_ph = tf.placeholder(tf.float32,shape=[None,3])
output = CNNmodel(img_ph)

W0711 06:50:28.357901 139820342941504 deprecation.py:323] From <ipython-input-5-a51b2f68b46c>:5: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0711 06:50:28.360522 139820342941504 deprecation.py:506] From /home/biswajit/anaconda3/envs/tfcpu/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0711 06:50:28.909855 139820342941504 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://githu

In [7]:
loss = tf.losses.softmax_cross_entropy(onehot_labels=label_ph,logits=output)
train = tf.train.AdamOptimizer(learning_rate=0.00002).minimize(loss)

W0711 06:50:29.263999 139820342941504 deprecation.py:323] From /home/biswajit/anaconda3/envs/tfcpu/lib/python3.7/site-packages/tensorflow/python/ops/losses/losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [8]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for i in range(10):
    los = 0
    imArr,lab = getNextBatch(sizeOfBatch=100,maxNumberOfCircles=3);
    _,los = sess.run([train,loss],feed_dict={img_ph:imArr,label_ph:lab})

    print("loss = "+ str(los) + "\trun = "+ str(i)+"\r")

loss = 1.0989671	run = 0
loss = 1.1759638	run = 1
loss = 1.0893501	run = 2
loss = 1.0878594	run = 3
loss = 1.0495259	run = 4
loss = 1.0551269	run = 5
loss = 1.1310538	run = 6
loss = 1.1187599	run = 7
loss = 1.1564925	run = 8
loss = 1.1111602	run = 9


In [9]:
imArr_test,lab_test = getNextBatch(sizeOfBatch=100,maxNumberOfCircles=3);
opval = sess.run([output],feed_dict={img_ph:imArr_test,label_ph:lab_test})
for i in range(99):
    print(np.asarray(opval)[0][i])
    print(lab_test[i])

[0.02794805 0.3916708  0.77321684]
[0. 0. 1.]
[0.02196407 0.76176244 0.29749358]
[1. 0. 0.]
[0.06095973 0.7462163  0.2102457 ]
[0. 0. 1.]
[0.05633527 0.74515975 0.25774026]
[1. 0. 0.]
[0.04940188 0.72529685 0.26531833]
[0. 1. 0.]
[0.04405388 0.8739834  0.19006339]
[0. 1. 0.]
[0.02932918 0.64371085 0.4147327 ]
[1. 0. 0.]
[0.01504442 0.7568348  0.49666056]
[0. 1. 0.]
[0.03205281 0.5073062  0.74414325]
[0. 0. 1.]
[0.03685045 0.7874962  0.18953118]
[1. 0. 0.]
[0.01982492 0.764563   0.3621658 ]
[1. 0. 0.]
[0.02187386 0.78003585 0.27453429]
[0. 1. 0.]
[0.0287315  0.8807967  0.24578115]
[0. 0. 1.]
[0.03642589 0.9072495  0.14840212]
[1. 0. 0.]
[0.06841901 0.89545345 0.108787  ]
[0. 0. 1.]
[0.06464055 0.9135265  0.10992229]
[0. 1. 0.]
[0.05162698 0.6215864  0.26730677]
[0. 1. 0.]
[0.0443691  0.8774302  0.12756741]
[0. 1. 0.]
[0.06120035 0.85130763 0.1381923 ]
[1. 0. 0.]
[0.05802631 0.6246407  0.38890052]
[1. 0. 0.]
[0.03880182 0.37654155 0.65300554]
[0. 0. 1.]
[0.01870012 0.77905226 0.2868522 ]

In [10]:
save_path ="./"
model_save=save_path+"model.ckpt"
saver=tf.train.Saver()
saver.save(sess,model_save) #saving the model
tf.train.write_graph(sess.graph_def, save_path, 'savegraph.pbtxt')

ResourceExhaustedError: model.ckpt.data-00000-of-00001.tempstate2178247038465557493; No space left on device
	 [[node save/SaveV2 (defined at <ipython-input-10-05284ff35edc>:3) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Errors may have originated from an input operation.
Input Source operations connected to node save/SaveV2:
 beta2_power (defined at <ipython-input-7-b486002a66c9>:2)	
 model/dense/bias (defined at <ipython-input-5-a51b2f68b46c>:13)	
 model/dense_1/kernel (defined at <ipython-input-5-a51b2f68b46c>:16)	
 model/conv2d_1/kernel (defined at <ipython-input-5-a51b2f68b46c>:8)	
 model/conv2d/bias (defined at <ipython-input-5-a51b2f68b46c>:5)

Original stack trace for 'save/SaveV2':
  File "/home/biswajit/anaconda3/envs/tfcpu/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/biswajit/anaconda3/envs/tfcpu/lib/python3.7/